# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from config import LensConfig
from crypto_factor_lens.weighting import WEIGHTING
from crypto_factor_lens.dataloader import DataLoader
from crypto_factor_lens.pipeline import CryptoFactorLens
from crypto_factor_lens.viz_plotly import PlotlyVisualizer
from crypto_factor_lens.regime_gmm import RegimeGMM
from crypto_factor_lens.factors import (
    MarketCapFactor,
    VolatilityFactor,
    MomentumFactor,
    DFTValueFactor,
    BetaPremiumFactor,
)

# Load Config and Data

In [3]:
# Load all settings from config.py (single source of truth)
cfg = LensConfig()
print(cfg)
# Load data
loader = DataLoader(cfg)
panel = loader.load_panel() 
panel.head()

LensConfig(cg_top_n=500, cg_sleep=0.2, data_dir='data/data_Last_4_Years', merged_csv='data/data_Last_4_Years/merged_sources.csv', preferred_price_col='price_cg', fallback_price_col='close_binance', preferred_volume_col='volume_cg', fallback_volume_col='volume_binance', use_history_filter=True, min_hist_window=30, use_liquidity_filter=False, liquidity_window=30, min_dollar_volume=100000.0, stable_known_symbols=['USDT', 'USDC', 'DAI', 'BUSD', 'FDUSD', 'TUSD', 'USDe', 'EURS', 'EURT', 'sUSD', 'EUR', 'EUROe', 'EURC', 'DOLA', 'USDD', 'CRVUSD', 'XAUT', 'PAXG', 'FDUSD', 'USDP', 'TUSD', 'BUSD'], stable_sigma_thresh=0.02, stable_corr_thresh=0.97, stable_window=30, stable_reference_symbol=['USDT', 'EURS'], vol_window=45, market_weights={'BTC': 0.6, 'ETH': 0.4}, universe_recalc='MS', universe_top_n=200, universe_mcap_roll_window=7, factor_rebalance_freq='2W', weighting='sqrt_mcap', dollar_neutral=True, market_beta_neutral=True, hedge_window=90, hedge_halflife=30)


price_cg  market_cap_cg     volume_cg  \
date       symbol                                            
2021-09-27 1INCH     2.681673   4.844438e+08  1.502934e+08   
           AAVE    292.943881   3.857114e+09  3.918999e+08   
           AB        0.000329   0.000000e+00  2.998118e+06   
           ADA       2.212884   7.235777e+10  3.307540e+09   
           AIOZ      0.273639   3.075618e+07  5.252990e+05   

                   circulating_supply_cg  total_supply_cg  max_supply_cg  \
date       symbol                                                          
2021-09-27 1INCH            1.397870e+09     1.500000e+09   1.500000e+09   
           AAVE             1.523307e+07     1.600000e+07   1.600000e+07   
           AB               7.898416e+10     9.882366e+10   1.000000e+11   
           ADA              3.652369e+10     4.500000e+10   4.500000e+10   
           AIOZ             1.186879e+09     1.205513e+09            NaN   

                   market_cap_rank_cg        cg_id_cg  open_binance  \
date       symbol                                                     
2021-09-27 1INCH                  245           1inch         2.679   
           AAVE                    43            aave       293.400   
           AB                     157  newton-project           NaN   
           ADA                     11         cardano         2.206   
           AIOZ                   249    aioz-network           NaN   

                   high_binance  low_binance  close_binance  volume_binance  \
date       symbol                                                             
2021-09-27 1INCH          2.950        2.513          2.524    1.544484e+07   
           AAVE         309.500      272.900        273.400    1.218325e+05   
           AB               NaN          NaN            NaN             NaN   
           ADA            2.277        2.128          2.130    1.509155e+08   
           AIOZ             NaN          NaN            NaN             NaN   

                  pair_binance quote_binance  
date       symbol                             
2021-09-27 1INCH     1INCHUSDT          USDT  
           AAVE       AAVEUSDT          USDT  
           AB             None          None  
           ADA         ADAUSDT          USDT  
           AIOZ           None          None

# Build Factor lens

In [4]:
# 3) Build factors
lens = CryptoFactorLens(cfg)
lens.load_from_panel(panel)
lens.build_factors() 

c:\Users\nahid\Projects\quant_backend\Factor_Lens\crypto_factor_lens\pipeline.py:72: FutureWarning: The 'fill_method' and 'limit' keywords in DataFrame.pct_change are deprecated and will be removed in a future version. Call ffill before calling pct_change instead.
  self.close_wide.pct_change(fill_method=None)


# Visualize Results

In [6]:
viz = PlotlyVisualizer(lens.factors)
viz.btc_series = lens.close_wide["BTC"]   # attach BTC prices
fig_cum = viz.cumulative(title="Factors vs BTC")
fig_cum.show()

In [7]:
# 5) Correlation heatmap
fig_corr = viz.factor_correlation(title="Crypto Factors - Correlation")
fig_corr.show()

In [9]:
btc = lens.close_wide["BTC"]
factors_df = lens.factors

# Fit a Gaussian Mixture Model to identify market regimes
gmm = RegimeGMM(n_regimes=3).fit(factors_df)

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
# Plot with BTC price series
fig_gmm = viz.plot_regimes(price=btc, regimes=gmm.labels_, title="Market Regimes Identified by GMM")
fig_gmm.show()

In [ ]:
fig_probs = viz.plot_regime_probabilities(btc, gmm.probs_)
fig_probs.show()

In [ ]:
# Fit a Gaussian Mixture Model to identify market regimes
gmm = RegimeGMM(n_regimes=2).fit(factors_df)

# Plot with BTC price series
fig_gmm = viz.plot_regimes(price=btc, regimes=gmm.labels_, title="Market Regimes Identified by GMM")
fig_gmm.show()

In [ ]:
fig_probs = viz.plot_regime_probabilities(btc, gmm.probs_)
fig_probs.show()